In [1]:
import pandas as pd
import glob 
import os 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
import pickle
from collections import defaultdict

#See # heatmap_collect.py

# Helpers

In [2]:
tar_file_OHSU = '/cluster/work/grlab/projects/projects2020_OHSU/share_OHUS_PNLL/Aug21_graph_data_current/OHSU_kmer_lists_Nov24.tar.gz'

base_folder_ETH = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/v2_v2.5f0752a_conf2_annotFrame_cap0_runs_pya0.17.1/TCGA_Breast_1102'

def ohsu_to_eth(path):
    cohort = {'NormalCohortcore_GTEx_': 'Gtexcore', 
             'NormalCohortpaired_': 'Matched', 
             'NormalCohortAll_':'Alls'}
    
    key_to_apply = [k for k in cohort if k in path]
    if key_to_apply: 
        key_to_apply = key_to_apply[0]
        path = path.replace(key_to_apply, '')
        path = path.replace('FiltNormalsC','FiltNormals{}C'.format(cohort[key_to_apply]) )

        path = path.replace('J_', 'G_')

        path = path.replace('CohortLim', '.0CohortLim')
        sample = path.split('_')[1]
        #print(path)
        return path, sample
    else:
        return None, None 

def get_eth_path(base_folder_ETH, name_eth=None, sample=None):
    path_o = None 
    if name_eth is not None: 
        path_list = os.path.join(base_folder_ETH, 'filter_' + sample, '*', name_eth, 'part*')
        path_list = glob.glob(path_list)
        if path_list:
            path_o = path_list[0]
    return path_o 
    

def set_stats(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    jaccard = float(intersection) / union
    return intersection, union, intersection/len(list1), intersection/len(list2), jaccard
     


def save_picke(i, sizes_ohsu, sizes_eth, sizes_intersect, 
              sizes_union, percent_ohsu_in_eth, percent_eth_in_ohsus, 
              jaccard):
    filehandler = open(b"/cluster/work/grlab/projects/projects2020_OHSU/plots/heatmaps/stats{}.pickle".format(i),"wb")
    res = [sizes_ohsu ,
    sizes_eth ,
    sizes_intersect, 
    sizes_union ,
    percent_ohsu_in_eth, 
    percent_eth_in_ohsus,
    jaccard ]
    pickle.dump(res,filehandler)

import tarfile
import pandas as pd

file_pair = {'eth':[], 'ohsu': []}
with tarfile.open(tar_file_OHSU, "r:*") as tar:
    file_names_OHSU = tar.getnames()

    for name_ohsu in file_names_OHSU:
        name_eth, sample = ohsu_to_eth(name_ohsu)
        eth_path = get_eth_path(base_folder_ETH, name_eth, sample)
        if (eth_path is not None) and os.path.isfile(eth_path):
            file_pair['eth'].append(eth_path) 
            file_pair['ohsu'].append(name_ohsu)
            


In [3]:
def path_to_condition(all_paths):
    # None handling added 
    
    all_paths = [path.replace('None', '0') for path in all_paths]
    sample_expr = [np.float(os.path.basename(path).split('ref_SampleLim')[1].split('Cohort')[0]) 
                   for path in all_paths ]

    foreground_cohort_expr = [np.float(os.path.basename(path).split('CohortLim')[1].split('Across')[0])
                                       for path in all_paths ]

    foreground_cohort_samples = [np.int(os.path.basename(path).split('Across')[1].split('_Filt')[0] ) 
                                 for path in all_paths ]

    background_cohort_expr = [np.float(os.path.basename(path).split('Cohortlim')[1].split('Across')[0])
                              for path in all_paths ]

    background_cohort_samples = [np.int(os.path.basename(path).split('Across')[2].split('_FiltUn')[0])
                                 for path in all_paths ]
    background_cohort_id = [os.path.basename(path).split('Normals')[1].split('lim')[0] for path in all_paths ]

    legend_quant = pd.DataFrame({'sample_expr':sample_expr, 
                  'foreground_cohort_expr': foreground_cohort_expr, 
                  'foreground_cohort_samples': foreground_cohort_samples, 
                  'background_cohort_expr': background_cohort_expr,
                  'background_cohort_samples' : background_cohort_samples, 
                    'background_cohort_id':background_cohort_id})


    sort_legend = False
    if sort_legend:
        legend_quant = legend_quant.sort_values("background_cohort_samples", ascending=False).\
        sort_values("background_cohort_expr", ascending=False).\
        sort_values("foreground_cohort_samples", ascending=False).\
        sort_values("foreground_cohort_expr", ascending=False).\
        sort_values("sample_expr", ascending=False)
    return legend_quant

In [4]:
def colors_from_values(values, palette_name):
    # normalize the values to range [0, 1]
    normalized = (values - min(values)) / (max(values) - min(values))
    # convert to indices
    indices = np.round(normalized * (len(values) - 1)).astype(np.int32)
    # use the indices to get the colors
    palette = sns.color_palette(palette_name, len(values))
    return np.array(palette).take(indices, axis=0)



In [ ]:
def plot_legend(y1, y2, y3, y4, y5, y6, x, save_path=None, save=False):
    #sns.set(style="white", context="talk")  
    sns.set_style("white")
    #, {
    #     "ytick.major.size": 0.1,
    #     "ytick.minor.size": 0.05,
    #     'grid.linestyle': '--', 
    #         "xtick.major.size": 0.1,
    #     "xtick.minor.size": 0.05,
    #     'grid.linestyle': '--'
    #  })
    rs = np.random.RandomState(8)  
    min_axes = -0.5
    y_label_size = 35
    x_label_size = 20
    # Set up the matplotlib figure  
    f, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(1, 6, figsize=(16,10), sharey=True)  
    ax1.invert_yaxis()

    sns.barplot(x=y1, y=x, palette=colors_from_values(y1, "YlOrRd"), ax=ax1, orient="h")  
    #ax1.axhline(0, color="k", clip_on=True)  
    #ax1.yaxis.set_minor_locator(ticker.MultipleLocator(1))
    ax1.set_xlim([min_axes, max(y1)])
    ax1.set_ylabel(y1.name, fontsize=y_label_size) 
    ax1.tick_params(labelsize=x_label_size) 


    sns.barplot(x=y2, y=x, palette=colors_from_values(y2, "vlag"), ax=ax2, orient="h", linewidth=1)  
    #ax2.axhline(0, color="k", clip_on=False)  
    ax2.set_xlim([min_axes, max(y2)])
    ax2.set_ylabel(y2.name, fontsize=y_label_size) 
    ax2.tick_params(labelsize=x_label_size) 

    sns.barplot(x=y3, y=x, palette=colors_from_values(y3, "deep"), ax=ax3, orient="h")
    #ax3.axhline(0, color="k", clip_on=False)  
    ax3.set_xlim([min_axes, max(y3)])
    ax3.set_ylabel(y3.name, fontsize=y_label_size) 
    ax3.tick_params(labelsize=x_label_size) 
    
    sns.barplot(x=y4, y=x, palette=colors_from_values(y4, "turbo"), ax=ax4, orient="h")
    #ax4.axhline(0, color="k", clip_on=False)  
    ax4.set_xlim([min_axes, max(y4)])
    ax4.set_ylabel(y4.name, fontsize=y_label_size) 
    ax4.tick_params(labelsize=x_label_size) 
    
    sns.barplot(x=y5, y=x, palette=colors_from_values(y5, "Accent"), ax=ax5, orient="h")
    #ax5.axhline(0, color="k", clip_on=False)  
    ax5.set_xlim([min_axes, max(y5)])
    ax5.set_ylabel(y5.name, fontsize=y_label_size) 
    ax5.tick_params(labelsize=x_label_size) 
    
    sns.scatterplot(x=np.ones(len(x)), y=x, ax=ax6, hue=y6)
    #ax6.axhline(0, color="k", clip_on=False)  
    #ax.legend(ax6.get_legend(),loc='lower right' ) 
    ax6.legend(loc='center right', bbox_to_anchor=(1,0))
    #ax6.invert_yaxis()
    ax6.set_ylabel(y6.name, fontsize=y_label_size) 
    ax6.tick_params(labelsize=x_label_size, labelbottom=False) 

    # Finalize the plot  
    sns.despine(bottom=True)  
    #plt.setp(ax.get_yticklabels(), fontsize=y_label_size, rotation=89)
    plt.setp(f.axes, yticks=np.arange(0,len(x),10), xlabel='')

    plt.tight_layout(h_pad=1) 
    ax1.invert_yaxis()
    if save:
        print("save to {}".format(save_path))
        plt.savefig(save_path, bbox_inches='tight', dpi = 300)
    plt.show()

# Specifics 

In [97]:
def wrapper_legend(all_paths, idx, save_path=None, save=False):
    all_paths = np.array(all_paths)[idx]
    prep_paths = ['/'.join(path.split('/')[:-1]) for path in all_paths]
    legend_quant = path_to_condition(prep_paths)
    print(legend_quant.shape)
    #Prepare Plot
    legend_quant.columns = [col.replace('_', ' ') for col in legend_quant.columns]
    y1, y2, y3, y4, y5, y6 = [legend_quant[col] for col in legend_quant.columns]
    legend_quant['index'] = np.arange(legend_quant.shape[0])
    x = legend_quant['index'] 


    plot_legend(y1, y2, y3, y4, y5, y6, x, save_path, save)

In [86]:
def heatmap2d(arr: np.ndarray,  save_path=None, save=False):
    #f, ax = plt.subplots(figsize=(7, 5))
    plt.imshow(arr, cmap='viridis')
    plt.colorbar()
    if save:
        print("save to {}".format(save_path))
        plt.savefig(save_path, bbox_inches='tight', dpi = 300)
    plt.show()

# Main Heatmap 

In [6]:
# Load Data
key_order = ['sizes_ohsu', 
'sizes_eth', 
'sizes_intersect', 
'sizes_union', 
'percent_ohsu_in_eth',
'percent_eth_in_ohsu', 
'jaccard']
heatmap_data = {}
for key in key_order:
    heatmap_data[key] = []
for i in np.arange(len(file_pair['eth'])):
    print(i)
    name = "/cluster/work/grlab/projects/projects2020_OHSU/plots/heatmaps/stats_single{}.pickle".format(i)
    data = pickle.load(open(name, 'rb'))
    for idx, key in enumerate(key_order):
        heatmap_data[key].append(data[idx])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
# Subset 
samples = ['TCGA-A2-A0D2-01A-21R-A034-07', 'TCGA-A2-A0SX-01A-12R-A084-07',
           'TCGA-AO-A0JM-01A-21R-A056-07',  'TCGA-BH-A18V-01A-11R-A12D-07', 
           'TCGA-C8-A12P-01A-11R-A115-07']
split_on_criteria =  defaultdict(list)
for idx, path in enumerate(file_pair['eth']):
    for sample in samples:
        if sample in path:
            split_on_criteria[sample].append(idx) 


In [93]:
save = False
plot_dir = '/cluster/work/grlab/projects/projects2020_OHSU/plots'
plot_dir = os.path.join(plot_dir, 'heatmaps', 'figs')
base_plot = 'v2_v2.5f0752a_conf2_BRCA'

In [ ]:
# Plot data 
for criteria, get_idx in split_on_criteria.items():
    print(criteria)
    base_sample =  base_plot + '_{}'.format(criteria)
    save_path = os.path.join(plot_dir, base_sample + '_hm_legend.png')  
    print(save_path)
    wrapper_legend(file_pair['eth'], get_idx, save_path, save)
    for key in key_order:
        if 'percent' in key:
            print(key)
            save_path = os.path.join(plot_dir, base_sample + '_{}.png'.format(key))
            print(save_path)
            heatmap2d(np.array(heatmap_data[key])[:,get_idx][get_idx,:].T, save_path, save)

